In [1]:
fff=!pwd
print(fff[0]+"/find_drugs_acting_agnostic_to_DEG.ipynb")

/workspace/02_cha_drug_repositioning/find_drugs_acting_agnostic_to_DEG.ipynb


In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [3]:
import numpy as np
import pandas as pd
import copy
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import warnings

from cmapPy.pandasGEXpress.parse import parse
from cmapPy.pandasGEXpress.write_gct import write
from cmapPy.pandasGEXpress.GCToo import GCToo
from cmapPy.pandasGEXpress.gctx2gct import gctx2gct_main

import torch

warnings.filterwarnings('ignore')
pd.set_option('display.float_format','{:.2f}'.format)

/root/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from super_classes import param_class as param_class_module
from utils import data_processing as data_processing_module
from utils import data_load as data_load_module
from run import train as train_module
from run import test_model as test_model_module

# Configure parameters

In [5]:
BASE_DIR="/workspace/datafiles/2023/04/02_cha_drug_repositioning/paper_data"
GCTX_PATH_GSE70138="GSE70138_Broad_LINCS_Level4_ZSPCINF_mlr12k_n345976x12328_2017-03-06.gctx"
GCTX_PATH_GSE92742="GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx"
TXT_META_PATH_GSE70138="GSE70138_Broad_LINCS_inst_info_2017-03-06.txt"
TXT_META_PATH_GSE92742="GSE92742_Broad_LINCS_inst_info.txt"
PERT_META_PATH_GSE70138="GSE70138_Broad_LINCS_pert_info_2017-03-06.txt"
PERT_META_PATH_GSE92742="GSE92742_Broad_LINCS_pert_info.txt"
TXT_LANDMARK_GENES_PATH="GSE92742_Broad_LINCS_gene_info_delta_landmark.txt"
MAKES_GROUP_DF_WITH_OUTLIER_DATA=False

Sent_FC_data="/workspace/datafiles/2023/04/02_cha_drug_repositioning/sent_data/DEG/GSE33814.csv"

In [6]:
configurable_parameters_dict={
  "BASE_DIR":BASE_DIR,
  "GCTX_PATH_GSE70138":GCTX_PATH_GSE70138,
  "GCTX_PATH_GSE92742":GCTX_PATH_GSE92742,
  "TXT_META_PATH_GSE70138":TXT_META_PATH_GSE70138,
  "TXT_META_PATH_GSE92742":TXT_META_PATH_GSE92742,
  "PERT_META_PATH_GSE70138":PERT_META_PATH_GSE70138,
  "PERT_META_PATH_GSE92742":PERT_META_PATH_GSE92742,
  "MAKES_GROUP_DF_WITH_OUTLIER_DATA":MAKES_GROUP_DF_WITH_OUTLIER_DATA
}

In [7]:
current_drug="memantine" # donepezil, galantamine, memantine

# Load AD candidates which have positive cosine similarity

In [8]:
AD_candidate_drugs_df=pd.read_csv('./result_files/cosine_similarities_wrt_{}.csv'.format(current_drug),encoding='utf-8')
AD_candidate_drugs_list=list(AD_candidate_drugs_df[AD_candidate_drugs_df['cosine_similarity']>0].iloc[1:,:]['drugs'].values)
AD_candidate_drugs_list[:10]

['xylazine',
 'betamethasone-acetate',
 'flunisolide',
 'fluticasone',
 'prednisolone',
 'heliotrine',
 'hydrocortisone-phosphate',
 'prednisolone-acetate',
 'prednisolone-hemisuccinate',
 'nomegestrol-acetate']

In [9]:
AD_candidate_drugs_df[AD_candidate_drugs_df['cosine_similarity']>0]

,drugs,cosine_similarity
0,memantine,1.00
1,xylazine,0.95
2,betamethasone-acetate,0.94
3,flunisolide,0.93
4,fluticasone,0.93
...,...,...
2130,lidamidine,0.00
2131,castanospermine,0.00
2132,elvitegravir,0.00
2133,EMD-53998,0.00


# Load data "after Concatenate two LINCS data"

In [10]:
with open('./result_files/labeling_samples_by_DEG_ranges/after_Concatenate_two_LINCS_data.pkl','rb') as f:
  expression_pert_df_cat=pickle.load(f)

In [11]:
expression_pert_df_cat

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,0.42,-0.22,0.93,0.02,0.00,-0.46,0.02,0.67,-0.34,0.63,...,-0.73,3.55,-7.93,-1.48,0.77,0.23,-0.89,-0.14,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,-0.17,-0.12,-0.10,0.46,-16.62,-0.43,0.75,1.35,-0.42,-1.32,...,-0.06,1.47,0.34,-2.37,-0.41,0.01,-0.41,13.45,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,-0.38,-0.34,0.36,-4.01,0.00,0.00,0.88,0.00,0.21,-1.22,...,0.93,0.33,0.11,-0.04,0.84,-0.94,1.58,12.14,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,0.09,-0.94,-0.08,-1.04,0.00,-0.33,1.50,0.67,-0.18,0.25,...,0.52,0.45,-0.56,0.26,-0.30,0.82,0.22,12.18,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,0.55,1.27,0.14,0.10,0.00,0.91,-0.76,0.49,0.29,0.47,...,-0.62,1.34,-0.10,-2.07,0.27,-1.65,-0.83,13.11,aminoguanidine,GSE70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.37,-2.10,0.20,0.88,-0.93,-0.05,-0.16,2.89,0.37,-0.05,...,-0.98,0.09,-2.55,0.40,1.16,0.64,-1.21,-0.23,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.38,-0.45,-0.01,-0.27,-0.17,0.10,-0.15,-2.07,-0.38,-1.23,...,-1.09,-2.10,-1.48,2.12,0.00,0.51,1.16,-0.09,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.55,0.00,-0.35,1.66,-0.84,-0.61,-0.49,-1.43,0.06,0.07,...,-1.04,-0.27,-1.21,-1.09,1.16,-0.37,-0.15,0.46,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.30,-0.67,1.18,0.53,-1.08,0.36,0.47,7.27,-0.48,-0.54,...,-0.96,-2.72,-0.77,0.38,0.27,0.22,-1.09,0.35,wortmannin,GSE92742


# Load DEG related data from `Drug repurposing using meta-analysis of gene expression in Alzheimer’s disease`

In [12]:
deg_list= ["ACTG1","AEBP1","AHNAK","ANKRD57","ANTXR2","ATP5H","ATP6V0C","ATP6V1E1","ATP6V1G2","BGN","C15ORF52","CDK2AP1","CHST6","CRTAP","CRYM","DDIT4L","DNALI1","ENO2","EZR","HEY2","HSPB2","HSPB3","ITGA10","ITGB5","ITPKB","KIAA0513","KIAA1467","KLC1","KLHL35","LOC100132707","NAP1L5","NINJ1","NRN1","NT5DC3","PCYOX1L","PDGFRB","PLCE1","PLXNB1","PRKX","PTBP1","PTRF","RGS7","RHOQ","SAP30L","SLC15A3","SLC6A12","SLC7A2","SPG20","SPHKAP","STAG1","STMN2","SVOP","SYDE1","SYN1","SYNGR1","TCEAL2","TEAD2","TMEM150A","TSPAN7","TUBB2A","ZNF438","ZNF621","SYN1","SYNGR1"]

# Get intersection between DEG and landmark genes

In [13]:
list(set(expression_pert_df_cat.columns).intersection(deg_list))

['CRTAP', 'ITGB5']

In [14]:
# CRTAP : up so down effect is needed
# ITGB5 : up so down effect is needed
target_DEG="CRTAP"
target_mechanism="down"

# Select DEG genes from landmark genes

In [15]:
expression_pert_df_cat

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,0.42,-0.22,0.93,0.02,0.00,-0.46,0.02,0.67,-0.34,0.63,...,-0.73,3.55,-7.93,-1.48,0.77,0.23,-0.89,-0.14,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,-0.17,-0.12,-0.10,0.46,-16.62,-0.43,0.75,1.35,-0.42,-1.32,...,-0.06,1.47,0.34,-2.37,-0.41,0.01,-0.41,13.45,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,-0.38,-0.34,0.36,-4.01,0.00,0.00,0.88,0.00,0.21,-1.22,...,0.93,0.33,0.11,-0.04,0.84,-0.94,1.58,12.14,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,0.09,-0.94,-0.08,-1.04,0.00,-0.33,1.50,0.67,-0.18,0.25,...,0.52,0.45,-0.56,0.26,-0.30,0.82,0.22,12.18,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,0.55,1.27,0.14,0.10,0.00,0.91,-0.76,0.49,0.29,0.47,...,-0.62,1.34,-0.10,-2.07,0.27,-1.65,-0.83,13.11,aminoguanidine,GSE70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.37,-2.10,0.20,0.88,-0.93,-0.05,-0.16,2.89,0.37,-0.05,...,-0.98,0.09,-2.55,0.40,1.16,0.64,-1.21,-0.23,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.38,-0.45,-0.01,-0.27,-0.17,0.10,-0.15,-2.07,-0.38,-1.23,...,-1.09,-2.10,-1.48,2.12,0.00,0.51,1.16,-0.09,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.55,0.00,-0.35,1.66,-0.84,-0.61,-0.49,-1.43,0.06,0.07,...,-1.04,-0.27,-1.21,-1.09,1.16,-0.37,-0.15,0.46,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.30,-0.67,1.18,0.53,-1.08,0.36,0.47,7.27,-0.48,-0.54,...,-0.96,-2.72,-0.77,0.38,0.27,0.22,-1.09,0.35,wortmannin,GSE92742


In [16]:
expression_pert_df_cat_selected_by_DEGs=expression_pert_df_cat[list(set(expression_pert_df_cat.columns).intersection(deg_list))+["pert_iname","LINCS"]]
expression_pert_df_cat_selected_by_DEGs

,CRTAP,ITGB5,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,-1.54,-0.42,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,1.29,0.04,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,0.44,-0.70,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,-0.16,-1.13,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,-0.20,-0.60,aminoguanidine,GSE70138
...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.94,1.33,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.85,2.28,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.61,2.01,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.94,0.85,wortmannin,GSE92742


# Create outlier removed LINCS

In [17]:
expression_pert_df_cat_selected_by_DEGs

,CRTAP,ITGB5,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,-1.54,-0.42,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,1.29,0.04,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,0.44,-0.70,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,-0.16,-1.13,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,-0.20,-0.60,aminoguanidine,GSE70138
...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.94,1.33,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.85,2.28,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.61,2.01,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.94,0.85,wortmannin,GSE92742


In [18]:
len(expression_pert_df_cat_selected_by_DEGs.pert_iname.unique())

20502

In [19]:
def loop_many(count_by_id_drugname):
  outlier_removed_df_list=[]
  for one_col in tqdm(count_by_id_drugname.columns):
    bound_lower=count_by_id_drugname[one_col].quantile(0.003)
    bound_upper=count_by_id_drugname[one_col].quantile(0.997)
    print('one_col',one_col)
    print('bound_lower',bound_lower)
    print('bound_upper',bound_upper)

    one_col_df=count_by_id_drugname[[one_col]]

    outlier_removed_col_df=one_col_df[(one_col_df[one_col]>=bound_lower)&(one_col_df[one_col]<=bound_upper)]
    # print('outlier_removed_col_df',outlier_removed_col_df)
    
    outlier_removed_df_list.append(outlier_removed_col_df)
  return outlier_removed_df_list

In [20]:
import multiprocessing

pool = multiprocessing.Pool(processes=1) # 현재 시스템에서 사용 할 프로세스 개수

df_split = np.array_split(expression_pert_df_cat_selected_by_DEGs.iloc[:,:-2],1,axis=1)
# df_split = np.array_split(expression_pert_df_cat,74)

bbb=pool.map(loop_many,df_split)
pool.close()
pool.join()

  0%|          | 0/2 [00:00<?, ?it/s]

one_col CRTAP
bound_lower -4.841760940551757
bound_upper 5.909094367027243


 50%|█████     | 1/2 [00:01<00:01,  1.30s/it]

one_col ITGB5
bound_lower -10.054645824432374
bound_upper 4.192511575222015


100%|██████████| 2/2 [00:02<00:00,  1.31s/it]


In [21]:
df_list=[]
for one_bbb in bbb:
  df_list.append(pd.concat(one_bbb,axis=1))

In [22]:
binary_df=pd.concat(df_list,axis=1)
binary_df

,CRTAP,ITGB5
REP.A001_A375_24H_X1_B22:A07,-1.54,-0.42
REP.A001_A375_24H_X1_B22:A08,1.29,0.04
REP.A001_A375_24H_X1_B22:A09,0.44,-0.70
REP.A001_A375_24H_X1_B22:A10,-0.16,-1.13
REP.A001_A375_24H_X1_B22:A11,-0.20,-0.60
...,...,...
PCLB003_PC3_24H_X3_B13:I04,NaN,-2.97
PCLB003_PC3_24H_X3_B13:I06,NaN,-2.65
PCLB003_PC3_24H_X3_B13:I14,NaN,1.31
PCLB003_PC3_24H_X3_B13:O02,NaN,0.00


In [23]:
df_inner=pd.merge(binary_df,expression_pert_df_cat[["pert_iname","LINCS"]],left_index=True,right_index=True,how='inner')
df_inner

,CRTAP,ITGB5,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,-1.54,-0.42,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,1.29,0.04,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,0.44,-0.70,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,-0.16,-1.13,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,-0.20,-0.60,aminoguanidine,GSE70138
...,...,...,...,...
PCLB003_PC3_24H_X3_B13:I04,NaN,-2.97,apicidin,GSE92742
PCLB003_PC3_24H_X3_B13:I06,NaN,-2.65,apicidin,GSE92742
PCLB003_PC3_24H_X3_B13:I14,NaN,1.31,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:O02,NaN,0.00,BRD-K64024097,GSE92742


In [24]:
display(df_inner.dropna(subset=['CRTAP'])[['CRTAP','pert_iname']])
display(len(df_inner.dropna(subset=['CRTAP'])[['CRTAP','pert_iname']]['pert_iname'].unique()))

,CRTAP,pert_iname
REP.A001_A375_24H_X1_B22:A07,-1.54,aminoguanidine
REP.A001_A375_24H_X1_B22:A08,1.29,aminoguanidine
REP.A001_A375_24H_X1_B22:A09,0.44,aminoguanidine
REP.A001_A375_24H_X1_B22:A10,-0.16,aminoguanidine
REP.A001_A375_24H_X1_B22:A11,-0.20,aminoguanidine
...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.94,wortmannin
PCLB003_PC3_24H_X3_B13:P21,0.85,wortmannin
PCLB003_PC3_24H_X3_B13:P22,0.61,wortmannin
PCLB003_PC3_24H_X3_B13:P23,0.94,wortmannin


20502

# Make train data

In [25]:
before_standardization_GSE70138=df_inner[df_inner['LINCS']=='GSE70138']
before_standardization_GSE92742=df_inner[df_inner['LINCS']=='GSE92742']

In [26]:
X_data_GSE70138=np.array(before_standardization_GSE70138.iloc[:,:-2], dtype=np.float)
y_data_GSE70138=np.array(before_standardization_GSE70138.iloc[:,-2:], dtype=np.object)
X_data_GSE92742=np.array(before_standardization_GSE92742.iloc[:,:-2], dtype=np.float)
y_data_GSE92742=np.array(before_standardization_GSE92742.iloc[:,-2:], dtype=np.object)

# Perform standardization

In [27]:
# only standardization separately
standard_scaler_GSE70138=StandardScaler()
X_data_standardized_GSE70138=standard_scaler_GSE70138.fit_transform(X_data_GSE70138)
standard_scaler_GSE92742=StandardScaler()
X_data_standardized_GSE92742=standard_scaler_GSE92742.fit_transform(X_data_GSE92742)

# Concat X and y again

In [28]:
expression_pert_np_copied_GSE70138=np.concatenate((X_data_standardized_GSE70138, y_data_GSE70138), axis=1)
expression_pert_np_copied_GSE92742=np.concatenate((X_data_standardized_GSE92742, y_data_GSE92742), axis=1)

# Data concatenation

In [29]:
expression_pert_np_copied=np.concatenate((expression_pert_np_copied_GSE70138,expression_pert_np_copied_GSE92742),axis=0)

In [30]:
expression_pert_df_copied=pd.DataFrame(expression_pert_np_copied)
expression_pert_df_copied

,0,1,2,3
0,-1.30,-0.10,aminoguanidine,GSE70138
1,1.10,0.18,aminoguanidine,GSE70138
2,0.38,-0.27,aminoguanidine,GSE70138
3,-0.13,-0.54,aminoguanidine,GSE70138
4,-0.17,-0.21,aminoguanidine,GSE70138
...,...,...,...,...
977485,NaN,-2.14,apicidin,GSE92742
977486,NaN,-1.90,apicidin,GSE92742
977487,NaN,1.12,wortmannin,GSE92742
977488,NaN,0.12,BRD-K64024097,GSE92742


In [31]:
expression_pert_df_copied.columns=expression_pert_df_cat_selected_by_DEGs.columns
expression_pert_df_copied

,CRTAP,ITGB5,pert_iname,LINCS
0,-1.30,-0.10,aminoguanidine,GSE70138
1,1.10,0.18,aminoguanidine,GSE70138
2,0.38,-0.27,aminoguanidine,GSE70138
3,-0.13,-0.54,aminoguanidine,GSE70138
4,-0.17,-0.21,aminoguanidine,GSE70138
...,...,...,...,...
977485,NaN,-2.14,apicidin,GSE92742
977486,NaN,-1.90,apicidin,GSE92742
977487,NaN,1.12,wortmannin,GSE92742
977488,NaN,0.12,BRD-K64024097,GSE92742


# Calculate mean expression value by grouping drug

In [32]:
expression_pert_df_cat_selected_by_DEGs

,CRTAP,ITGB5,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,-1.54,-0.42,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,1.29,0.04,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,0.44,-0.70,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,-0.16,-1.13,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,-0.20,-0.60,aminoguanidine,GSE70138
...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.94,1.33,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.85,2.28,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.61,2.01,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.94,0.85,wortmannin,GSE92742


In [33]:
grouped_df_w_outlier=expression_pert_df_cat_selected_by_DEGs.groupby('pert_iname').mean()
grouped_df_wo_outlier=expression_pert_df_copied.groupby('pert_iname').mean()

# Calculate up and down-regulated thresholds (25% and 75%)

In [34]:
# up_and_down_thresholds=grouped_df[target_DEG].describe()
# up_and_down_thresholds

q_low=grouped_df_wo_outlier[target_DEG].quantile(0.16)
q_hig=grouped_df_wo_outlier[target_DEG].quantile(0.68)
print('q_low,q_hig',q_low,q_hig)

q_low,q_hig -0.2744914723132836 0.13760631707484405


In [35]:
# group data with outlier : q_low,q_hig -0.3428348340889755 0.14699343738129605
# group data without outlier : q_low,q_hig -0.2744914723132836 0.13760631707484405

# Filter candidates by DEG agnostic threshold

In [36]:
entire_drug_names=list(grouped_df_w_outlier.index)
print(entire_drug_names[:3])

['(+)-3-(1-propyl-piperidin-3-yl)-phenol', '(+/-)-7-hydroxy-2-(N,N-di-n-propylamino)tetralin', '1,2,3,4,5,6-hexabromocyclohexane']


In [37]:
filtered_drug_list=[]
filtered_drug_expression_list=[]
filtered_drug_threshold_list=[]
# for one_target_drug in AD_candidate_drugs_list:
for one_target_drug in entire_drug_names:
  expression_val=grouped_df_w_outlier[target_DEG][one_target_drug]
  if target_mechanism=='up':
    # threshold_up=up_and_down_thresholds['75%']
    threshold_up=q_hig
    if expression_val>=threshold_up:
      filtered_drug_list.append(one_target_drug)
      filtered_drug_expression_list.append(expression_val)
      filtered_drug_threshold_list.append(threshold_up)
  elif target_mechanism=='down':
    # threshold_down=up_and_down_thresholds['25%']
    threshold_down=q_low
    if expression_val<=threshold_down:
      filtered_drug_list.append(one_target_drug)
      filtered_drug_expression_list.append(expression_val)
      filtered_drug_threshold_list.append(threshold_down)

In [38]:
filtered_candidates={}
filtered_candidates['filtered_drug']=filtered_drug_list
filtered_candidates['filtered_drug_expression']=filtered_drug_expression_list
filtered_candidates['filtered_drug_threshold']=filtered_drug_threshold_list

In [39]:
pd.DataFrame(filtered_candidates).sort_values(by=['filtered_drug_expression'], axis=0, ascending=True)

,filtered_drug,filtered_drug_expression,filtered_drug_threshold
3867,bruceantin,-6.00,-0.27
4072,romidepsin,-3.29,-0.27
1570,BRD-K41350512,-2.78,-0.27
3967,homoharringtonine,-2.57,-0.27
807,BRD-K20624574,-2.57,-0.27
...,...,...,...
826,BRD-K21179706,-0.27,-0.27
1770,BRD-K46597784,-0.27,-0.27
3800,SD-6-035-A7,-0.27,-0.27
2398,BRD-K64034190,-0.27,-0.27


In [40]:
AD_candidate_drugs_having_positive_cosine_similarity_df=AD_candidate_drugs_df[AD_candidate_drugs_df['cosine_similarity']>0]
AD_candidate_drugs_having_positive_cosine_similarity_df

,drugs,cosine_similarity
0,memantine,1.00
1,xylazine,0.95
2,betamethasone-acetate,0.94
3,flunisolide,0.93
4,fluticasone,0.93
...,...,...
2130,lidamidine,0.00
2131,castanospermine,0.00
2132,elvitegravir,0.00
2133,EMD-53998,0.00


In [41]:
df_final_candidates=pd.DataFrame(filtered_candidates)
df_final_candidates['current_drug']=current_drug
df_final_candidates['DEG']=target_DEG
display(df_final_candidates)
df_final_candidates_with_cosine_similarity=df_final_candidates.merge(AD_candidate_drugs_having_positive_cosine_similarity_df,left_on='filtered_drug',right_on='drugs',how='inner')
df_final_candidates_with_cosine_similarity=df_final_candidates_with_cosine_similarity.sort_values(by=['cosine_similarity'], axis=0, ascending=False)
df_final_candidates_with_cosine_similarity=df_final_candidates_with_cosine_similarity.drop(columns=['drugs'])
display(df_final_candidates_with_cosine_similarity)
display(df_final_candidates_with_cosine_similarity.shape)

,filtered_drug,filtered_drug_expression,filtered_drug_threshold,current_drug,DEG
0,2-aminobenzenesulfonamide,-0.28,-0.27,memantine,CRTAP
1,2-aminotetralin,-0.55,-0.27,memantine,CRTAP
2,2-methoxyestradiol,-0.36,-0.27,memantine,CRTAP
3,3-matida,-0.28,-0.27,memantine,CRTAP
4,4-hydroxy-2-nonenal,-0.29,-0.27,memantine,CRTAP
...,...,...,...,...,...
4122,vorinostat,-1.11,-0.27,memantine,CRTAP
4123,wiskostatin,-0.39,-0.27,memantine,CRTAP
4124,withaferin-a,-0.54,-0.27,memantine,CRTAP
4125,z-leu3-VS,-0.32,-0.27,memantine,CRTAP


,filtered_drug,filtered_drug_expression,filtered_drug_threshold,current_drug,DEG,cosine_similarity
83,quinethazone,-0.40,-0.27,memantine,CRTAP,0.88
30,SB-269970,-0.30,-0.27,memantine,CRTAP,0.87
55,cyclophosphamide,-0.30,-0.27,memantine,CRTAP,0.87
9,GR-46611,-0.28,-0.27,memantine,CRTAP,0.81
42,amoxicillin,-0.45,-0.27,memantine,CRTAP,0.77
...,...,...,...,...,...,...
86,scriptaid,-0.69,-0.27,memantine,CRTAP,0.01
75,nelarabine,-0.30,-0.27,memantine,CRTAP,0.01
94,vorinostat,-1.11,-0.27,memantine,CRTAP,0.00
92,trequinsin,-0.29,-0.27,memantine,CRTAP,0.00


(96, 6)

In [43]:
fn='./result_files/find_drugs_acting_agnostic_to_DEG/candidates_filtered_by_DEG_agnostic_{}_{}.csv'.format(current_drug,target_DEG)
df_final_candidates_with_cosine_similarity.to_csv(fn,sep=',',encoding='utf-8',index=False)